In [1]:
import sys
import os

# Replace 'path_to_your_module' with the absolute or relative path to the target folder
target_path = '/ibex/ai/home/liz0l/codes/depth-fm'
if target_path not in sys.path:
    sys.path.append(target_path)


In [2]:
import os
from tqdm import tqdm
import argparse
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import torch
import matplotlib

import torch.nn.functional as F
from src.models.amodalsynthdrive.depth_anything_v2_raw.dpt import DepthAnythingV2
from src.util.image_util import chw2hwc, colorize_depth_maps
from src.util.alignment import align_depth_least_square

from src.util import metric
from src.util.metric import MetricTracker
import einops

from src.util.logging_util import tb_logger, eval_dic_to_text
from src.util.config_util import (
    find_value_in_omegaconf,
    recursive_load_config,
)

from src.models import get_model
from torchvision.transforms import InterpolationMode, Resize


In [3]:
model = get_model('PartialCompletionContentDPT', loss_stategy='entire_target_object')
model = model.from_pretrained('/ibex/ai/home/liz0l/codes/depth-fm/work_dir/project_folder/cvpr_baseline/jo/20241024_194215/checkpoint/iter_060000', strict=True)
# model = get_model('ADDeepLab', encoder_name='resnet50', channels=[64, 256, 512, 1024, 2048], up_sample_channels=[64, 128, 256, 512, 1024])
# model = model.from_pretrained('/ibex/ai/home/liz0l/codes/depth-fm/work_dir/project_folder/cvpr_baseline/deeplab/20241024_142321/checkpoint/iter_060000', strict=True)
model.eval()
model.cuda()

<All keys matched successfully>
<All keys matched successfully>
Loading weights from local directory


PartialCompletionContentDPT(
  (model): DPTDepthModel(
    (pretrained): Module(
      (model): VisionTransformer(
        (patch_embed): PatchEmbed(
          (proj): Conv2d(4, 1024, kernel_size=(16, 16), stride=(16, 16))
          (norm): Identity()
        )
        (pos_drop): Dropout(p=0.0, inplace=False)
        (patch_drop): Identity()
        (norm_pre): Identity()
        (blocks): Sequential(
          (0): Block(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): Attention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (q_norm): Identity()
              (k_norm): Identity()
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): Identity()
            (drop_path1): Identity()
            (norm2): LayerNorm((1024,), eps=1e-

In [4]:
# prepare path
image_path = '/ibex/ai/home/liz0l/codes/Marigold/data/sam/SA-1B-Downloader/images' # raw image
occ_image_path = '/ibex/ai/home/liz0l/codes/depth-fm/data/sam/pix2gestalt_occlusions_release/occlusion'
visible_mask_path = '/ibex/ai/home/liz0l/codes/depth-fm/data/sam/pix2gestalt_occlusions_release/visible_object_mask'
whole_mask_path = '/ibex/ai/home/liz0l/codes/depth-fm/data/sam/pix2gestalt_occlusions_release/whole_mask' # gt amodal mask
whole_path = '/ibex/ai/home/liz0l/codes/depth-fm/data/sam/pix2gestalt_occlusions_release/whole'
combined_depth_path = '/ibex/ai/home/liz0l/codes/depth-fm/data/sam/pix2gestalt_occlusions_release/depth_da_update_combine' # gt depth (aligned)
occ_depth_path = '/ibex/ai/home/liz0l/codes/depth-fm/data/sam/pix2gestalt_occlusions_release/depth_da_update_occ' # depth with occ
custom_amodal_mask_path = 'work_dir/project_folder/cvpr_base_pix2gestalt_results/amodal_mask'

In [5]:
# split file
# valid_samples = [
#     'images/sa_10537752.jpg depth/10537752_depth.png',
#     'images/sa_10072253.jpg depth/10072253_depth.png',
#     'images/sa_8535471.jpg depth/8535471_depth.png',
#     'images/sa_6333511.jpg depth/6333511_depth.png',
# ]
valid_samples = [
    'images/sa_1460103.jpg depth/1460103_depth.png',
]



output_path = '/ibex/ai/home/liz0l/codes/depth-fm/work_dir/project_folder/cvpr_baseline/jo/20241024_194215'
# output_path = '/ibex/ai/home/liz0l/codes/depth-fm/work_dir/project_folder/cvpr_baseline/deeplab/20241024_142321'

output_amodal_depth = os.path.join(output_path, 'amodal_depth')
os.makedirs(output_amodal_depth, exist_ok=True)

In [6]:
resize_to_hw = [512, 512]
resize_transform = Resize(size=resize_to_hw, interpolation=InterpolationMode.NEAREST_EXACT)

In [7]:
id = valid_samples[0].split("_")[1].split(".")[0]

# load combined image
image_file_path = os.path.join(occ_image_path, "{}_occlusion.png".format(id))
image = Image.open(image_file_path) # 256
image = np.transpose(image, (2, 0, 1)).astype(int)
image = torch.from_numpy(np.asarray(image))
image = image / 255
image = resize_transform(image) # 518


# load invisible mask
visible_mask = os.path.join(visible_mask_path, "{}_visible_mask.png".format(id))
visible_mask = Image.open(visible_mask)  #256
visible_mask = torch.from_numpy(np.asarray(visible_mask)).unsqueeze(0)
visible_mask = resize_transform(visible_mask) # 518

# load depth gt
gt_depth_file_path = os.path.join(combined_depth_path, "{}_depth.png".format(id))
gt_depth = Image.open(gt_depth_file_path) # 512
gt_depth = torch.from_numpy(np.asarray(gt_depth)).unsqueeze(0) 
gt_depth = gt_depth / 65535
gt_depth = resize_transform(gt_depth) # 518

# load depth observation
observation_depth_file_path = os.path.join(occ_depth_path, "{}_depth.png".format(id))
observation_depth = Image.open(observation_depth_file_path) # 256
observation_depth = torch.from_numpy(np.asarray(observation_depth)).unsqueeze(0)
observation_depth = observation_depth / 65535
observation_depth = resize_transform(observation_depth) # 518
# Image.fromarray(depth_colored).save('./work_dir/debug.png')

# use gt amodal mask to cal metric
whole_mask = os.path.join(whole_mask_path, "{}_whole_mask.png".format(id))
whole_mask = Image.open(whole_mask) 
whole_mask = torch.from_numpy(np.asarray(whole_mask)).unsqueeze(0)
whole_mask = resize_transform(whole_mask) # 518
whole_mask = whole_mask > 0

input_image = image.cuda()
depth = model(
            input_image.unsqueeze(dim=0), 
            guide_rgb=None, 
            guide_mask=(whole_mask.unsqueeze(dim=0).float().cuda() * 2) - 1,
            observation=(observation_depth.unsqueeze(dim=0).float().cuda() * 2) - 1) # both: from -1 to 1

# _, depth = model(
#             input_image.unsqueeze(dim=0), 
#             guide_rgb=None, 
#             guide_mask=(whole_mask.unsqueeze(dim=0).float().cuda() * 2) - 1,
#             observation=(observation_depth.unsqueeze(dim=0).float().cuda() * 2) - 1) # both: from -1 to 1

depth = depth.squeeze().detach().cpu()
pred_np = depth.detach().cpu().numpy()
depth_save = (depth.numpy() * 65535.0).astype(np.uint16)
Image.fromarray(depth_save).save('{}/{}_depth.png'.format(output_amodal_depth, id), mode="I;16")


depth_align, scale, shift = align_depth_least_square(
    gt_arr=observation_depth, # utilize the observation!
    pred_arr=pred_np,
    valid_mask_arr=visible_mask.detach().cpu().bool(), # only use the visible part to do this alignment
    return_scale_shift=True,
    max_resolution=None)

depth_save = (depth_align * 65535.0).astype(np.uint16)
Image.fromarray(depth_save).save('{}/{}_depth_aligned.png'.format(output_amodal_depth, id), mode="I;16")